<a href="https://colab.research.google.com/github/RobertoAlessandri/SaveTheWorld/blob/main/MediaPipeCopiedCode1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install mediapipe

import mediapipe as mp
import cv2
import time
import numpy as np
import pandas as pd
import os
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils # For drawing keypoints
points = mpPose.PoseLandmark # Landmarks

## here is where it will be needed to change to path
# we'll need a dir with our positions
path = "DATASET/TRAIN/plank" # enter dataset path
data = []
for p in points:
  x = str(p)[13:]
  data.append(x + "_x")
  data.append(x + "_y")
  data.append(x + "_z")
  data.append(x + "_vis")
data = pd.DataFrame(columns = data) # Empty dataset
# ^Here the columns include the thirty-three key points that will be detected by
# the blaze pose detector (each key point contains four attributes).
# x and y are normalized form 0 to 1
# z represent landmark depth with hips as the origin
# visibility = probability that the landmark is visible in the image

     |████████████████████████████████| 32.7 MB 56.3 MB/s 


In [4]:
count = 0

# iterate through the pose images individually, extracting the key points using
# the blaze pose model and sotirng them in temporary array "temp"
for img in os.listdir(path):

  temp = []

  img = cv2.imread(path + "/" + img)

  imageWidth, imageHeight = img.shape[:2]

  imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  blackie = np.zeros(img.shape) # Blank image

  results = pose.process(imgRGB)

  if results.pose_landmarks:

    # mpDraw.draw_landmarks(img, results.pose_landmarks, mpPose.POSE_CONNECTIONS) #draw landmarks on image

    mpDraw.draw_landmarks(blackie, results.pose_landmarks, mpPose.POSE_CONNECTIONS) # draw landmarks on blackie

    landmarks = results.pose_landmarks.landmark

    for i,j in zip(points,landmarks):

      temp = temp + [j.x, j.y, j.z, j.visibility]

    data.loc[count] = temp

    count +=1

  cv2.imshow("Image", img)

  cv2.imshow("blackie",blackie)

  cv2.waitKey(100)

# We append this temporary array as a new record in our dataset
data.to_csv("dataset3.csv") # save the data as a csv file

# Blaze pose model takes RGB images, OpenCV BGR

# target value = label for the ML model


FileNotFoundError: ignored

In [ ]:
## Classification

from sklearn.svm import SVC
data = pd.read_csv("dataset3.csv")
# we train the dataset built on SVC with the target variable as the Y label
X,Y = data.iloc[:,:132],data['target']
model = SVC(kernel = 'poly')
model.fit(X,Y)
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils
path = "enter image path" # !!!
img = cv2.imread(path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
results = pose.process(imgRGB)
if results.pose_landmarks:
  landmarks = results.pose_landmarks.landmark
  for j in landmarks:
    temp = temp + [j.x, j.y, j.z, j.visibility]
  y = model.predict([temp])
  if y == 0:
    asan = "plank"
  else:
    asan = "goddess"
  print(asan)
  cv2.putText(img, asan, (50,50), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),3)
  cv2.imshow("image",img)

